<a href="https://colab.research.google.com/github/tozanni/Data_Science_Notebooks/blob/main/CIFAR10_Autoencoder_CNN_BaselineReto_SinSolucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras import datasets, layers, models
from keras.utils import np_utils
from keras import regularizers
from keras.layers import Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
import numpy as np

Cargar el dataset cifar10 en un conjunto inicial de training (imagenes y labels) y test (imágenes y labels)

In [ ]:
(x_train0, y_train0), (x_test0, y_test0) = datasets.cifar10.load_data()


In [ ]:
# Creamos la lista de etiquetas CIFAR10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

### 1. Pre-procesamiento del dataset

a. Convierte los pixeles de las imágenes del conjunto de train y test a tipo float

b. Normaliza, dividiendo el valor de todos los pixeles por 255, ya que es el valor máximo de intensidad de cada pixel.

c. Codifica las etiquetas y_train y y_test como one-hot utilizando la funcion to_categorical


In [ ]:
# Convierte los pixeles de las imágenes del conjunto de train y test a tipo float

# Normaliza, dividiendo el valor de todos los pixeles por 255, ya que es el valor máximo de intensidad de cada pixel.

# Codifica las etiquetas y_train y y_test como one-hot utilizando la funcion to_categorical


Dividimos el test set en 7000 imagenes de validacion (x_val_images) y 3000 de test set (x_test_images)


In [ ]:
# Dividir test en validacion y test, 7000 de validacion y 3000 de test
x_val_images = x_test_images[:7000]
x_test_images = x_test_images[7000:]

Dividimos las etiquetas de validacion (y_val y y_test) de tal forma que correspondan a los conjuntos previos.

In [ ]:
# Dividir etiquetas de validacion
y_val_labels = y_test_labels[:7000]
y_test_labels = y_test_labels[7000:]

In [ ]:
# Algunos imports complementarios
from keras.layers import Input, Dense, Dropout, Activation, Add, Concatenate, Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, MaxPool2D, Flatten, BatchNormalization
import keras.backend as K
from keras.models import Model

In [ ]:
# Modelo encoder-decoder de referencia

input_img = Input(shape=(32, 32, 3))

# Red encoder
x = Conv2D(64, (3, 3), padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Red decoder
x = Conv2D(16, (3, 3), padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
decoded = Activation('sigmoid')(x)


In [ ]:
# Modelo Encoder-Decoder
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        18464 

### 2. Entrenamiento de autoencoder

a. Entrena el modelo no-supervisado encoder-decoder de tal forma que el modelo aprenda a reconstruir su propio input, con batch_size 64, mínimo 20 épocas. 

b. Investiga e implementa el uso de los callbacks de EarlyStopping y ModelCheckpoint

c. Visualización del proceso de entrenamiento. Grafica Training Loss vs. Validation Loss.


d. Visualización del input reconstruido

Imprime un conjunto de imágenes originales y comparalas con la imagen reconstruida por el autoencoder. 

Utiliza las siguientes funciones de referencia.

In [ ]:
c10test = model.predict(x_test_images)
c10val = model.predict(x_val_images)

In [ ]:
# Funcion para mostrar imagenes originales y reconstruidas

def showOrigDec(orig, dec, num=10):
    import matplotlib.pyplot as plt
    n = num
    plt.figure(figsize=(20, 4))

    for i in range(n):
        # original
        ax = plt.subplot(2, n, i+1)
        plt.imshow(orig[i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # reconstruidas
        ax = plt.subplot(2, n, i +1 + n)
        plt.imshow(dec[i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()
  

In [ ]:
# Muestra algunas imagenes originales y reconstruidas utilizando tu funcion
#showOrigDec(x_test_images, c10test)


A continuación generaremos el modelo encoder del autoencoder. Este modelo lo utilizarás para obtener la representación codificada de los conjuntos originales y con ella entrenar un clasificador. 

In [ ]:
# Modelo Encoder
encoder = Model(input_img, encoded)
encoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        1846

### 3. Extracción de features del autoencoder

Utilizando el **método predict del modelo encoder** extrae las variables que te indicamos a continuación: 

a. Codifica el el conjunto de imágenes de entrenamiento utilizando el método predict del encoder, y guárdalo en una variable llamada **gist_train_ae**

b. Codifica el conjunto de imágenes de validación, utilizando el método predict del encoder y guárdalo en una variable llamada **gist_valid_ae**

c. Codifica el conjunto de imágenes de prueba utilizando el método predict del encoder y guárdalo en una variable llamada **gist_test_ae**


In [ ]:
# Completa esta función...

# gist_train_ae = 
# gist_valid_ae = 
# gist_test_ae = 


A continuación definimos un clasificador con una capa convolucional y dos capas densas, que aprenderá a clasificar el input una vez procesado por el codificador. 

Puedes utilizar el siguiente clasificador como referencia:

In [ ]:
input = Input((gist_train_ae.shape[1], gist_train_ae.shape[2], gist_train_ae.shape[3]))

x = Conv2D(64, 3, padding="same")(input)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(2)(x)
x = Dropout(0.5)(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

output = Dense(num_classes, activation='softmax')(x)

decoder_classifier = Model(input, output)
decoder_classifier.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['acc'])
decoder_classifier.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 4, 4, 16)]        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 4, 4, 64)          9280      
                                                                 
 activation_7 (Activation)   (None, 4, 4, 64)          0         
                                                                 
 batch_normalization_7 (Batc  (None, 4, 4, 64)         256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 2, 2, 64)         0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 2, 2, 64)          0   

### 4. Entrenamiento y evaluación del clasificador

a. Entrena el clasificador **decoder_classifier** definido en la fase previa utilizando como input la representación codificada del training set obtenida en el paso previo y como output los labels originales del conjunto de training.

b. Calcula la pérdida de validación del modelo, utilizando la representación codificada de los datos de validación y como ouput los labels originales del conjunto de validación.


c. Genera predicciones con el modelo clasificador, utiliza el conjunto de test codificado modelo encoder. Guárdalo en la variable pred.




In [ ]:
# Obtener las etiquetas con el clasificador
#pred = 
# print(pred)

# Convertimos las predicciones a una lista de etiquetas única
pred_classes = np.argmax(pred, axis=1)
print(pred_classes)


d. Evalúa las predicciones del modelo y obten la matriz de confusión.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

cm=confusion_matrix(y_test_labels.argmax(axis=1), pred.argmax(axis=1))
print("Classification Report:\n")
cr=classification_report(y_test_labels.argmax(axis=1), pred.argmax(axis=1), target_names=class_names)
print(cr)

Classification Report:

              precision    recall  f1-score   support

    airplane       0.54      0.56      0.55       296
  automobile       0.50      0.64      0.56       311
        bird       0.41      0.27      0.32       299
         cat       0.33      0.35      0.34       302
        deer       0.51      0.28      0.37       278
         dog       0.45      0.31      0.37       293
        frog       0.55      0.57      0.56       302
       horse       0.58      0.53      0.55       298
        ship       0.59      0.68      0.64       314
       truck       0.43      0.65      0.51       307

    accuracy                           0.49      3000
   macro avg       0.49      0.48      0.48      3000
weighted avg       0.49      0.49      0.48      3000



### 5. Mejoras a los modelos

Aplicando los conceptos del curso modifica dichas redes para obtener un mejor accuracy ¿Crees poder lograr un 70% o tal vez 80% de accuracy con tu modelo? OJO: NO está permitido modificar el modelo clasificador. 

a.    Experimenta agregando capas, modificando operaciones y modificando las dimensiones de las capas actuales. OJO: Recuerda que para que tu modelo encoder-decoder siga funcionando y puedas reconstruir las imágenes codificadas, las capas de MaxPool del encoder deben de corresponder a las capas UpSample del decoder. Tip: ¿Las capas de pooling ayudan o perjudican a tu modelo?
 
b.    En una celda de texto, justifica los cambios realizados, a la arquitectura.
 
c.     Genera la matriz de confusión de tu ensamble de modelos mejorado. Recuerda que debes re-entrenar el clasificador si la arquitectura del autoencoder cambia. 
